In [1]:
import autogen
import requests
import json
import dotenv
import os

from splunk_functions import splunk_query

dotenv.load_dotenv(".env")

True

In [2]:
splunk_host="localhost"
port="8089"
username=os.getenv('SPLUNK_UN')
password=os.getenv('SPLUNK_PW')

In [3]:
config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt-4-0314", "gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
    },
)

In [14]:
START_DATE = "07/31/2017:20:15"
END_DATE = "08/31/2017:18:00"

from datetime import datetime, timedelta

In [15]:
splunk_time_format = '%m/%d/%Y:%H:%M'
start_date = datetime.strptime(START_DATE, splunk_time_format)
end_date = datetime.strptime(END_DATE, splunk_time_format)
one_day = timedelta(days=1)
end_less_one = end_date - one_day
end_less_one_str = datetime.strftime(end_less_one, splunk_time_format)

In [20]:
end_less_one.isoformat()

'2017-08-30T18:00:00'

In [21]:
sourcetypes = splunk_query("index=botsv2 | stats values(sourcetype)", latest_time=end_date.isoformat(), earliest_time=end_less_one.isoformat()).split("\n0")[-1]

In [30]:
sourcetypes.split("\n0")[-1]

'  MSAD:NT6:Health MSAD:NT6:SiteInfo Perfmon:CPU Perfmon:LogicalDisk Perfmon:Memory Perfmon:NTDS Perfmon:Network Perfmon:Network_Interface Perfmon:PhysicalDisk Perfmon:Process Perfmon:Processor Perfmon:System Powershell:ScriptExecutionSummary Script:ListeningPorts Unix:ListeningPorts Unix:UserAccounts WinHostMon WinRegistry access_combined apache:error apache_error auditd bandwidth collectd cpu csp-violation df interfaces iostat lastlog linux_audit linux_secure mysql:connection:stats mysql:instance:stats mysql:server:stats mysql:status mysql:tableStatus mysql:table_io_waits_summary_by_index_usage mysql:transaction:details mysql:transaction:stats mysql:variables netstat openPorts osquery_info osquery_results osquery_warning package pan:system pan:traffic protocol ps stream:arp stream:dhcp stream:dns stream:http stream:icmp stream:ip stream:ldap stream:mysql stream:smb stream:smtp stream:tcp stream:udp suricata symantec:ep:agent:file symantec:ep:agt_system:file symantec:ep:packet:file sy

In [4]:
ASSISTANT_SYSTEM_MESSAGE = """You are a helpful AI assistant.
Solve tasks using your coding and language skills.
In the following cases, suggest python code (in a python coding block), shell script (in a sh coding block), or Splunk spl for the user to execute.
    1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.
    2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.

Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or perform any other action beyond executing the code you suggest. The user can't modify your code. So do not suggest incomplete code which requires users to modify. Don't use a code block if it's not intended to be executed by the user.
If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block as the first line. Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
When you find an answer, verify the answer carefully. Include verifiable evidence in your response if possible.
Reply "TERMINATE" in the end when everything is done.
"""


USERAGENT_SYSTEM_MESSAGE = """
You are a helpful AI assistant.
Solve tasks using your coding and language skills.
"""

In [5]:
# The REST API endpoint URL
url = f'https://{splunk_host}:8089/services/data/indexes?output_mode=json'

# Make a GET request to the Splunk REST API
response = requests.get(url, auth=(username, password), verify=False)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()
    
    # Extract the index names
    index_names = [entry['name'] for entry in data['entry']]

indices="\n".join(index_names)

/home/bots/BOTS/bots/lib/python3.11/site-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [6]:
FUNCTIONS = [
    {
        "name": "splunk_query",
        "description": "Use this function to query Splunk spl. Input should be a fully formed spl query.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": f"""
                            SPL query extracting info to answer the user's question.
                            Splunk has the following indexes:
                                {indices}
                            The query should be returned in plain text, not in JSON.
                            """,
                },
            #     "earliest_time": {
            #         "type": "string",
            #         "description": "The earliest time for the search (default last 24 hours). Input should be in SPL format, e.g. '-24h@h'"
            #     },
            #    "latest_time": {
            #         "type": "string",
            #         "description": "The latest time for the search (default now). Input should be in SPL format, e.g.'now'"
            #     } 
            },
            "required": ["query"],
        },
    }
]

In [7]:
# create an AssistantAgent named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    system_message=ASSISTANT_SYSTEM_MESSAGE,
    llm_config={
        "seed": 42,  # seed for caching and reproducibility
        "config_list": config_list,  # a list of OpenAI API configurations
        "temperature": 0,  # temperature for sampling,
        "functions": FUNCTIONS
    },  # configuration for autogen's enhanced inference API which is compatible with OpenAI API
)
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,  # set to True or image name like "python:3" to use docker
    },
)

user_proxy.register_function(
    function_map={
        'splunk_query': splunk_query
    }
)

In [8]:
# the assistant receives a message from the user_proxy, which contains the task description
user_proxy.initiate_chat(
    assistant,
    message="""Find the most common completed search sting made today.""",
)


user_proxy (to assistant):

Find the most common completed search sting made today.

--------------------------------------------------------------------------------
assistant (to user_proxy):

To find the most common completed search string made today, we can use the Splunk SPL (Search Processing Language). We will query the `_audit` index, which contains information about searches. We will filter for events where the search was completed (`action=search` and `info=completed`), and then count the occurrences of each search string (`search` field). Finally, we will sort the results in descending order and return the most common search string.

Here is the SPL query:

```spl
index=_audit action=search info=completed earliest=@d latest=now() 
| stats count by search 
| sort count desc 
| head 1
```

Let's execute this query using the `splunk_query` function.
***** Suggested function Call: splunk_query *****
Arguments: 
{
  "query": "index=_audit action=search info=completed earliest=@d l

In [9]:
print(assistant.last_message()['content'])

The most common completed search string made today is:

```spl
| rest /services/search/distributed/groups splunk_server=local | fields + title, member | where isnotnull(mvfind(member,"localhost:localhost")) | eval peerURI="localhost" | rename title as search_groups | fields + peerURI, search_groups | mvcombine delim=" " search_groups
```

This search string was executed 7 times today.

TERMINATE
